**S&P 500 Stocks Data Preprocessing**

In [ ]:
!pip install ta

  Preparing metadata (setup.py) ... done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29411 sha256=7c52a480010890be911c01398bc849a48ee05eb099d868ac16b9a6388889fba3
  Stored in directory: /root/.cache/pip/wheels/5f/67/4f/8a9f252836e053e532c6587a3230bc72a4deb16b03a829610b
Successfully built ta


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import pandas as pd
import ta
from sklearn.preprocessing import MinMaxScaler
import numpy as np
sp500_stocks = pd.read_csv('/content/sp500_stocks.csv')


In [ ]:
sp500_stocks['Date'] = pd.to_datetime(sp500_stocks['Date'], errors='coerce')
sp500_stocks.sort_values(by=['Symbol', 'Date'], inplace=True)

In [ ]:
sp500_stocks.shape

(1816836, 8)

In [ ]:
sp500_stocks.head()

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume
32508,2010-01-04,A,20.434929,22.389128,22.625179,22.267525,22.453505,3815561.0
32509,2010-01-05,A,20.212959,22.145924,22.331903,22.002861,22.324751,4186031.0
32510,2010-01-06,A,20.141132,22.067240,22.174536,22.002861,22.067240,3243779.0
32511,2010-01-07,A,20.115025,22.038628,22.045780,21.816881,22.017166,3095172.0
32512,2010-01-08,A,20.108498,22.031473,22.067240,21.745352,21.917025,3733918.0


In [ ]:
nat_rows = sp500_stocks[sp500_stocks['Date'].isna()]


In [ ]:
def calculate_indicators(df):
    if len(df) > 30:
        indicator_macd = ta.trend.MACD(close=df['Close'], window_slow=26, window_fast=12, window_sign=9)
        indicator_rsi = ta.momentum.RSIIndicator(close=df['Close'], window=30)
        indicator_cci = ta.trend.CCIIndicator(high=df['High'], low=df['Low'], close=df['Close'], window=30)
        indicator_dx = ta.trend.ADXIndicator(high=df['High'], low=df['Low'], close=df['Close'], window=30)

        df['macd'] = indicator_macd.macd()
        df['rsi_30'] = indicator_rsi.rsi()
        df['cci_30'] = indicator_cci.cci()
        df['dx_30'] = indicator_dx.adx()
    else:
        df['macd'] = df['rsi_30'] = df['cci_30'] = df['dx_30'] = 0

    return df

In [ ]:
sp500_stocks['returns'] = sp500_stocks.groupby('Symbol')['Close'].pct_change()
processed_stocks = sp500_stocks.groupby('Symbol').apply(calculate_indicators).reset_index(drop=True)
returns_pivot = processed_stocks.pivot_table(index='Date', columns='Symbol', values='returns')
returns_pivot = returns_pivot.dropna(axis=1, how='any')

mean_returns = returns_pivot.mean()
covariance_returns = returns_pivot.cov()
inv_covmat = np.linalg.pinv(covariance_returns)

def calculate_turbulence(x, mean_vec, inv_cov_matrix):
    x_minus_mu = x - mean_vec
    return np.dot(np.dot(x_minus_mu.T, inv_cov_matrix), x_minus_mu)


In [ ]:
turbulence_index = returns_pivot.apply(lambda row: calculate_turbulence(row, mean_returns, inv_covmat), axis=1)
processed_stocks = processed_stocks.join(turbulence_index.rename('Turbulence'), on='Date')



In [ ]:
print(processed_stocks.columns)


Index(['Date', 'Symbol', 'Adj Close', 'Close', 'High', 'Low', 'Open', 'Volume',
       'returns', 'macd', 'rsi_30', 'cci_30', 'dx_30', 'Turbulence'],
      dtype='object')


In [ ]:
processed_stocks

,Date,Symbol,Adj Close,Close,High,Low,Open,Volume,returns,macd,rsi_30,cci_30,dx_30,Turbulence
0,2010-01-04,A,20.434929,22.389128,22.625179,22.267525,22.453505,3815561.0,NaN,NaN,NaN,NaN,0.0,NaN
1,2010-01-05,A,20.212959,22.145924,22.331903,22.002861,22.324751,4186031.0,-0.010863,NaN,NaN,NaN,0.0,669.913227
2,2010-01-06,A,20.141132,22.067240,22.174536,22.002861,22.067240,3243779.0,-0.003553,NaN,NaN,NaN,0.0,699.266296
3,2010-01-07,A,20.115025,22.038628,22.045780,21.816881,22.017166,3095172.0,-0.001297,NaN,NaN,NaN,0.0,728.067842
4,2010-01-08,A,20.108498,22.031473,22.067240,21.745352,21.917025,3733918.0,-0.000325,NaN,NaN,NaN,0.0,552.403419
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1816831,2024-05-03,ZTS,167.070007,167.070007,171.690002,166.949997,170.449997,3007000.0,-0.000957,-1.549334,48.960019,82.688555,NaN,723.103981
1816832,2024-05-06,ZTS,165.949997,165.949997,167.619995,164.880005,167.350006,3081300.0,-0.006704,-0.899451,48.208770,60.710501,NaN,381.617128
1816833,2024-05-07,ZTS,168.449997,168.449997,172.229996,166.470001,166.960007,3337500.0,0.015065,-0.180604,49.980995,89.217102,NaN,533.909369
1816834,2024-05-08,ZTS,164.919998,164.919998,167.500000,161.000000,167.080002,4673100.0,-0.020956,0.103058,47.601728,44.420038,NaN,359.145480


In [ ]:
processed_stocks.drop('Adj Close',axis=1,inplace=True)

In [ ]:
scaler = MinMaxScaler()
feature_columns = ['Close', 'Volume', 'macd', 'rsi_30', 'cci_30', 'dx_30']
processed_stocks[feature_columns] = scaler.fit_transform(processed_stocks[feature_columns])

In [ ]:
processed_stocks.fillna(0, inplace=True)
processed_stocks.replace([float('inf'), float('-inf')], 0, inplace=True)


In [ ]:
processed_stocks['Date'] = pd.to_datetime(processed_stocks['Date'])
processed_stocks["day"] = processed_stocks["Date"].dt.dayofweek

processed_stocks = processed_stocks.rename(columns={"Symbol": "tic", "Date": "Date"})

processed_stocks = processed_stocks.sort_values(by=["Date", "tic"]).reset_index(drop=True)

In [ ]:
processed_stocks = processed_stocks.rename(columns={"Close": "close", "Date": "date", "High": "high", "Low":"low", "Open":"open","Volume": "volume","Turbulence":"turbulence"})

In [ ]:
processed_stocks.head()

,date,tic,close,high,low,open,volume,returns,macd,rsi_30,cci_30,dx_30,turbulence,day
0,2010-01-04,A,0.002678,22.625179,22.267525,22.453505,0.002028,0.0,0.0,0.0,0.0,0.0,0.0,0
1,2010-01-04,AAL,0.000503,4.940000,4.660000,4.840000,0.005230,0.0,0.0,0.0,0.0,0.0,0.0,0
2,2010-01-04,AAPL,0.000857,7.660714,7.585000,7.622500,0.262483,0.0,0.0,0.0,0.0,0.0,0.0,0
3,2010-01-04,ABBV,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0
4,2010-01-04,ABNB,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0


In [ ]:
processed_stocks.to_csv('/content/drive/MyDrive/final_processed.csv', index=False)
